In [2]:
# -*- coding: utf-8 -*-
# 題目 8-2：重力改正與異常（GRS80，FAC=0.3087691 mGal/m，Bouguer slab 2πGρh）
# 只用標準函式庫；會印出結果並輸出 HTML 報告在當前資料夾

import math, os, datetime

# ===== 輸入 =====
lat_deg = 48.1195          # 緯度 (°N)
elev_m  = 487.9            # 高程或水深 (m)
obs_mgal = 980_717.39      # 觀測重力 (mGal)

rho_rock  = 2670.0         # 岩石密度 kg/m^3 (2.67 g/cc)
rho_water = 1030.0         # 海水密度 kg/m^3 (1.03 g/cc)

# ===== 常數 / 模型 =====
# GRS80 正常重力
gamma_e = 9.7803267715
k = 0.00193185138639
e2 = 0.00669437999013

def normal_gravity_mgal(phi_deg: float) -> float:
    phi = math.radians(phi_deg)
    s2 = math.sin(phi)**2
    gamma_mps2 = gamma_e * (1 + k*s2) / math.sqrt(1 - e2*s2)
    return gamma_mps2 * 1e5  # 轉 mGal

# 自由空氣梯度
FA_gradient = 0.3087691  # mGal/m

# Bouguer 板改正（2πGρh），回傳「改正值大小」（mGal）
G = 6.67430e-11
def bouguer_correction_mgal(thickness_m: float, density_kgm3: float) -> float:
    return 2 * math.pi * G * density_kgm3 * thickness_m * 1e5

# ===== 計算 =====
gamma_mgal = normal_gravity_mgal(lat_deg)

# (a) 陸上：高程 h
FAC_land = FA_gradient * elev_m                  # 在公式中為 + 號
BC_land  = bouguer_correction_mgal(elev_m, rho_rock)  # 在公式中為 − 號
dFA_land = obs_mgal - gamma_mgal + FAC_land
dB_land  = dFA_land - BC_land

# (b) 海面：水深 = elev_m（站在海面上，FAC=0；加入水的布格改正）
FAC_ocean = 0.0
BC_ocean  = bouguer_correction_mgal(elev_m, rho_water)  # 在公式中為 + 號
dFA_ocean = obs_mgal - gamma_mgal
dB_ocean  = dFA_ocean + BC_ocean

# ===== 終端輸出 =====
print("=== 題目 8-2 計算（GRS80）===")
print(f"理論重力 γ(φ)            = {gamma_mgal:,.3f} mGal")
print(f"(a) 自由空氣改正 FAC(+)   = {FAC_land:,.3f} mGal")
print(f"(a) 布格改正 岩石板(−)     = {BC_land:,.3f} mGal")
print(f"(a) 自由空氣異常 Δg_FA(陸) = {dFA_land:,.3f} mGal")
print(f"(a) 布格異常   Δg_B(陸)   = {dB_land:,.3f} mGal")
print(f"(b) 自由空氣異常 Δg_FA(海) = {dFA_ocean:,.3f} mGal")
print(f"(b) 海水布格改正(+)        = {BC_ocean:,.3f} mGal")
print(f"(b) 布格異常   Δg_B(海)   = {dB_ocean:,.3f} mGal")

# ===== 產生 HTML 報告 =====
rows = [
    ("輸入 (Input)","緯度 (°N)", f"{lat_deg:.6f}"),
    ("輸入 (Input)","高度/水深 (m)", f"{elev_m:.1f}"),
    ("輸入 (Input)","觀測重力 g_obs (mGal)", f"{obs_mgal:,.2f}"),
    ("常數 (Const)","自由空氣梯度 (mGal/m)", f"{FA_gradient:.7f}"),
    ("常數 (Const)","ρ_rock (kg/m³)", f"{rho_rock:.0f}"),
    ("常數 (Const)","ρ_water (kg/m³)", f"{rho_water:.0f}"),
    ("(a) 理論重力 γ(φ)","(mGal)", f"{gamma_mgal:,.3f}"),
    ("(a) 自由空氣改正 FAC (+)","(mGal)", f"{FAC_land:,.3f}"),
    ("(a) 布格改正 岩石板 (−)","(mGal)", f"{BC_land:,.3f}"),
    ("(a) 自由空氣異常 Δg_FA(陸)","(mGal)", f"{dFA_land:,.3f}"),
    ("(a) 布格異常 Δg_B(陸)","(mGal)", f"{dB_land:,.3f}"),
    ("(b) 自由空氣異常 Δg_FA(海)","(mGal)", f"{dFA_ocean:,.3f}"),
    ("(b) 海水布格改正 (+)","(mGal)", f"{BC_ocean:,.3f}"),
    ("(b) 布格異常 Δg_B(海)","(mGal)", f"{dB_ocean:,.3f}"),
]

html = f"""<!doctype html>
<html lang="zh-Hant">
<meta charset="utf-8">
<title>題目 8-2：重力改正與異常（Python 報告）</title>
<style>
body{{font-family:-apple-system,BlinkMacSystemFont,'Segoe UI',Noto Sans TC,Roboto,Helvetica,Arial;
      margin:2rem;line-height:1.6}}
table{{border-collapse:collapse;width:100%;margin:1rem 0}}
th,td{{border:1px solid #ddd;padding:8px;text-align:left}}
th{{background:#fafafa}}
.note{{background:#fff8e1;border:1px solid #ffe0a3;padding:.8rem;border-radius:8px}}
</style>
<h1>題目 8-2：重力改正與異常</h1>
<p>產生時間：{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}</p>
<h2>結果總表</h2>
<table>
<thead><tr><th>項目</th><th>單位/說明</th><th>數值</th></tr></thead>
<tbody>
"""
for a,b,c in rows:
    html += f"<tr><td>{a}</td><td>{b}</td><td>{c}</td></tr>\n"

html += f"""</tbody></table>
<div class="note">
<b>公式（符號採用）</b><br>
Δg_FA(陸)=g_obs−γ(φ)+FAC；　Δg_B(陸)=g_obs−γ(φ)+FAC−BC_rock。<br>
Δg_FA(海)=g_obs−γ(φ)；　　　Δg_B(海)=g_obs−γ(φ)+BC_water。<br>
FAC=0.3087691·h；　BC=2πGρh（mGal）。
</div>
</html>"""

out_path = os.path.abspath("gravity_8-2_solution.html")
with open(out_path, "w", encoding="utf-8") as f:
    f.write(html)

print("\nHTML 報告已輸出：", out_path)



=== 題目 8-2 計算（GRS80）===
理論重力 γ(φ)            = 980,901.781 mGal
(a) 自由空氣改正 FAC(+)   = 150.648 mGal
(a) 布格改正 岩石板(−)     = 54.630 mGal
(a) 自由空氣異常 Δg_FA(陸) = -33.743 mGal
(a) 布格異常   Δg_B(陸)   = -88.372 mGal
(b) 自由空氣異常 Δg_FA(海) = -184.391 mGal
(b) 海水布格改正(+)        = 21.074 mGal
(b) 布格異常   Δg_B(海)   = -163.317 mGal

HTML 報告已輸出： /workspaces/Homework-7_calculate_gravity/gravity_8-2_solution.html
